In [36]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

In [37]:
dataset = load_dataset('json', data_files='/Users/mikaumana/Downloads/dataset.json')
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b")
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-3b")


  0%|          | 0/1035 [02:33<?, ?it/s]


Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.35s/it]


In [38]:
dataset['train'][0]

{'question_id': 0,
 'db_id': 'california_schools',
 'question': 'What is the highest eligible free rate for K-12 students in the schools in Alameda County?',
 'evidence': 'Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`',
 'SQL': "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1",
 'difficulty': 'simple',
 'database_schema': 'CREATE TABLE `frpm` (\n  CDSCode TEXT PRIMARY KEY REFERENCES schools(CDSCode),\n  Academic Year TEXT,\n  County Code TEXT,\n  District Code INTEGER,\n  School Code TEXT,\n  County Name TEXT,\n  District Name TEXT,\n  School Name TEXT,\n  District Type TEXT,\n  School Type TEXT,\n  Educational Option Type TEXT,\n  NSLP Provision Status TEXT,\n  Charter School (Y/N) INTEGER,\n  Charter School Number TEXT,\n  Charter Funding Type TEXT,\n  IRC INTEGER,\n  Low Grade TEXT,\n  High Grade TEXT,\n  Enrollment (K-12

In [39]:
train_dataset = dataset['train'].train_test_split(test_size=0.1)["train"]
eval_dataset = dataset['train'].train_test_split(test_size=0.1)["test"]

In [40]:
def formatting_prompts_func(train_dataset):
  output_texts = []
  for i in range(len(train_dataset['question'])):
    question = train_dataset['question'][i]
    query = train_dataset['SQL'][i]
    database_schema = train_dataset['database_schema'][i]
    user_message = f"""Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ```sql and ``` tags.
{database_schema}
###
Question: {question}

```sql
{query} ;
```
<|EOT|>
"""
    output_texts.append(user_message)
  return output_texts

In [41]:
x = formatting_prompts_func(train_dataset)

In [51]:
tokenized_prompts = []
tokenizer.pad_token = tokenizer.eos_token
for row in x:
    inputs = tokenizer(row, padding="max_length", truncation=True,  max_length=512, return_tensors="pt").to(model.device)
    tokenized_prompts.append(inputs)

In [43]:
tokenized_prompts[0]

{'input_ids': tensor([[15768,   253,  1563, 11700,  7180,    13,   634,  2628,   310,   281,
          6635,   253, 39542,   614, 11700,  7316,  1677,   253,  2608,   434,
          1953,    15,   187, 12501,   634,  3662,  3304,   253,  2634,  5190,
         11296,   285,  2634,  5190, 14610,    15,   187, 25875, 22454,  2634,
           925,  2617,    65,   313,   187, 50276,  3717,  4061,   853, 46434,
          4653, 44825, 25111,  5689, 38064, 20712,  1410,  6629,     9,  3717,
          4061,   853,   582,   187, 50276, 42228,   280, 10519, 46434,    13,
           187, 50276, 43582,  6307, 46434,    13,   187, 50276, 31711,  6307,
         41847,    13,   187, 50276, 28014,  6307, 46434,    13,   187, 50276,
         43582,  9424, 46434,    13,   187, 50276, 31711,  9424, 46434,    13,
           187, 50276, 28014,  9424, 46434,    13,   187, 50276, 31711,  8078,
         46434,    13,   187, 50276, 28014,  8078, 46434,    13,   187, 50276,
         25735,  1050, 27357,  8078, 4

In [44]:
training_args = TrainingArguments(
    output_dir="./results",          # Output directory
    eval_strategy="epoch",     # Evaluation strategy (per epoch)
    learning_rate=5e-5,              # Learning rate
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay
    logging_dir="./logs",            # Directory for logs
    save_total_limit=3,              # Number of model checkpoints to save
)

In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_prompts,  # Training dataset
)

In [53]:
trainer.train()

  0%|          | 0/1035 [00:00<?, ?it/s]

RuntimeError: expand(torch.FloatTensor{[4, 4, 1, 1, 512]}, size=[4, 1, 1, 512]): the number of sizes provided (4) must be greater or equal to the number of dimensions in the tensor (5)